# Dataframe Columns

Understanding the dataframe columns:
- `ts_recv`: time our client received the message in UTC
- `ts_event`: exchange event timestamp of when the trade acc happened
- `rtype`: message type code
- `publisher_id`: exchange ID
- `instrument_id`: symbol of the product
- `action`: update action
    - `A`: add
    - `M`: modify
    - `D`: delete
    - `T`: trade
- `side`: which side the update refers to B for buy, A for ask
- `depth`: book level for this single update (0 for top of book, 1 for next level, ...) 
- `price`: price in update
- `size`: number of shares in update
- `flags`: special conditions
- `ts_in_delta`: time since the previous message
- `sequence`: monotonic sequence number for ordering

There are columns labled `bid_px_00/ask_px_00` which gives all top-10 levels simultaneously from 00 to 09. 00 is the best bid/ask. 

# Preprocessing Inputs
- use `publisher_id` as venue key and the fields `ask_px_00` and `ask_sz_00` as the venues best asks and displayed size (side=ask, depth=0)

- for every unique `ts_event` keep only the first message per `publisher_id` that gives one level-1 snapshots per vanue per timestmap
    - feed those snapshots order by `ts_event` into the backtest

# Example

## Data Cleaning

``` python
raw = [
  {ts:"13:00", pid:1, ask_px:100, ask_sz:50},
  {ts:"13:00", pid:1, ask_px:101, ask_sz:45},
  {ts:"13:00", pid:2, ask_px:102, ask_sz:40},
  {ts:"13:01", pid:2, ask_px:103, ask_sz:30},
  {ts:"13:01", pid:1, ask_px:101, ask_sz:75},
]
```
This is an example raw message that the `l1_day.csv` will feed us. The dataset contains duplicates which will need to be preprocessed. First sort this by `ts_event` and thne deduplicate by keeping the first row. This will yield the best ask/bid in the data stream.

``` python
snapshots = [
  {
    ts: "13:00",
    venues: [
      {"venue":1, "ask_px":100, "ask_sz":50},
      {"venue":2, "ask_px":102, "ask_sz":40}
    ]
  },
  {
    ts: "13:01",
    venues: [
      {"venue":1, "ask_px":101, "ask_sz":75},
      {"venue":2, "ask_px":103, "ask_sz":30}
    ]
  }
]
```
The snapshots contain the timestamp of the best asks for each venue. A venue is a `publisher_id` in the `l1_data.csv`. Let say that we must split 200 shares in 100 share increments across the different potential buers. We must choose a split that minimizes cost.

## Static Allocator

This is where the static allocator comes. For two venue in the example (multiples of 100 for 200 shares):
```python
splits = [
    [0,200],
    [100,100],
    [200,0]
]
```
For each split ($[x_1,x_2]$) for venues $v_1$ and $v_2$, the cost is defined by the function:
```python
cost = ∑ ( xᵢ * ask_pxᵢ
           + λ_over  * max(0, xᵢ − ask_szᵢ)
           + λ_under * max(0, ask_szᵢ − xᵢ)
           + θ_queue * (1/ask_szᵢ) )
```
The optimal split is $[200,0]$ which has the cheapest cost of 20,018.05


## Backtest The Strategy

A backtest loop is run on the the snapshots where:
- initialize variables: `remaining_cash = 200, cash_spent = 0`
- at snapshot "13:00":
    - allocator send 200 to venue 1, 0 to venue 2
    - venue 1 has 50 avaliable: fills 50 @ 100
    - `remaining_cash=150`
- at snapshot "13:01":
    - allocator now splits 150: $[0,150], [100,50], [150,0]$
    - venue 1 has 75 avaliable: fills 75 @ 101
    - `remaining_cash=75`
- continue until `remaining_cash=0` or data ends

## Finding Best Parameter

I will do a parametr search for the best parameters and pick the combination yielding minimum total `cash_spent`
```python
λ_over ∈ [0.01, 0.1, 1.0]
λ_under ∈ [0.01, 0.1, 1.0]
θ_queue ∈ [0, 10, 100]
```

## Three Baselines Comparison
Using the same snapshots I will run the baselines of:
1. best-ask: always hit the lowest price venue first
2. 60s TWAP: break total order into equal sized slices and executes each slice at regular time intervals regardless of how much vol is trading for. For exmaple, to buy 1000 shares over 10 mins, you place 100 shares every minute
3. VWAP: executes your roder in proportion to that market's avaliable liquidity meaning that you trade more when displayed volume is high and less when it's low

*To aid understanding, this example shows what the code does or for reviewing*


In [ ]:
import pandas as pd
import numpy as np
import json
from datetime import datetime
import math
from dataclasses import dataclass
from pathlib import Path
from typing import Dict, List, Tuple


In [25]:
## hold quote information from single venue
@dataclass(slots=True)
class VenueLevel:
    venue_id: int # id of venue
    px: float     # price offered
    sz: int       # size offered at price

## snapshot holds venue quotes at a particular timestamp
@dataclass(slots=True)
class Snapshot:
    ts: str # timestamp of the snapshot
    venues: List[VenueLevel] # list of VenueLevel objects

    @staticmethod
    def from_group(ts: str, group: pd.DataFrame) -> "Snapshot":
        return Snapshot(
            ts,
            [
                VenueLevel(int(v), float(p), int(s))
                for v, p, s in zip(
                    group.publisher_id.values,
                    group.ask_px_00.values,
                    group.ask_sz_00.values,
                )
            ],
        )

## load the CSV and parse into snapshots sorted by timestamp 
def load_snapshots(csv_path: str | Path) -> List[Snapshot]:
    csv_path = Path(csv_path)
    if not csv_path.exists():
        raise FileNotFoundError(csv_path)
    df = (
        pd.read_csv(csv_path)
        .sort_values("ts_event")
        .drop_duplicates(subset=["ts_event", "publisher_id"], keep="first")
        .reset_index(drop=True)
    )
    return [Snapshot.from_group(ts, grp) for ts, grp in df.groupby("ts_event", sort=True)]

## Generate n tuple (in multiples of chunk) summing to total
def enumerate_splits(total: int, n: int, *, chunk: int = 100) -> np.ndarray:
    """All non-negative n-tuples (multiples of *chunk*) summing to *total*."""
    levels = total // chunk + 1
    grids = [np.arange(levels) * chunk for _ in range(n)]
    mesh = np.array(np.meshgrid(*grids, indexing="ij"))
    combos = mesh.reshape(n, -1).T
    return combos[combos.sum(axis=1) == total]

## generate n-tiples for each split allocaiton across venues
def compute_cost_matrix(
    splits: np.ndarray,
    venues: List[VenueLevel],
    lambda_over: float,
    lambda_under: float,
    theta_queue: float,
) -> np.ndarray:
    if len(splits) == 0:
        return np.empty(0)

    px = np.fromiter((v.px for v in venues), float)
    q = np.fromiter((v.sz for v in venues), float)

    x = splits
    exec_pay = (x * px).sum(axis=1)
    over_pen = lambda_over * np.maximum(0, x - q).sum(axis=1)
    under_pen = lambda_under * np.maximum(0, q - x).sum(axis=1)
    queue_pen = theta_queue * (1 / np.where(q > 0, q, 1)).sum()
    return exec_pay + over_pen + under_pen + queue_pen

## main allocator: determine how much to send to each venue for a snapshot
def allocate(
    remaining: int,
    venues: List[VenueLevel],
    lambda_over: float,
    lambda_under: float,
    theta_queue: float,
    *,
    chunk: int = 100,
) -> Dict[int, int]:
    """Return {venue_id: shares_to_send} for the current snapshot."""
    if remaining == 0 or len(venues) == 0:
        return {}

    splits = enumerate_splits(remaining, len(venues), chunk=chunk)

    # retry with finest granularity if coarse grid produced nothing
    if len(splits) == 0 and chunk != 1:
        splits = enumerate_splits(remaining, len(venues), chunk=1)

    # ultimate fall-back: hit the cheapest venue with everything
    if len(splits) == 0:
        best = min(venues, key=lambda v: v.px)
        return {best.venue_id: remaining}

    costs = compute_cost_matrix(splits, venues, lambda_over, lambda_under, theta_queue)
    best_split = splits[costs.argmin()]
    return {v.venue_id: int(x) for v, x in zip(venues, best_split)}

# run the backtest with given parameters across all snapshots
def run_backtest(
    snapshots: List[Snapshot],
    lambda_over: float,
    lambda_under: float,
    theta_queue: float,
    *,
    order_size: int = 5_000,
    side: str = "buy",
    chunk: int = 100,
) -> Tuple[float, float, List[Tuple[str, float]]]:
    sign = 1 if side == "buy" else -1
    remaining, cash, filled = order_size, 0.0, 0
    cumulative: List[Tuple[str, float]] = []

    for snap in snapshots:
        if remaining <= 0:
            break
        alloc = allocate(
            remaining, snap.venues,
            lambda_over, lambda_under, theta_queue,
            chunk=chunk,
        )
        for v in snap.venues:
            send = alloc.get(v.venue_id, 0)
            fill = min(send, v.sz)
            cash += sign * fill * v.px
            filled += fill
        remaining = order_size - filled
        cumulative.append((snap.ts, abs(cash)))
    avg = abs(cash) / filled if filled else float("inf")
    return abs(cash), avg, cumulative

# baseline strategy: always hit the best ask (lowest price)
def best_ask_baseline(snaps: List[Snapshot], *, size=5_000):
    remain, cash = size, 0.0
    for s in snaps:
        if remain <= 0:
            break
        v = min(s.venues, key=lambda v: v.px)
        take = min(remain, v.sz)
        cash += take * v.px
        remain -= take
    return cash, cash / size, None

# vwap
def vwap_baseline(snaps: List[Snapshot], *, size=5_000):
    num = sum(v.px * v.sz for s in snaps for v in s.venues)
    den = sum(v.sz for s in snaps for v in s.venues)
    vwap = num / den
    return vwap * size, vwap, None

# twap
def twap_baseline(snaps: List[Snapshot], *, size=5_000, bucket=60):
    times = [datetime.fromisoformat(s.ts[:-1]) for s in snaps]
    start = times[0]
    buckets: Dict[int, List[Snapshot]] = {}
    for t, s in zip(times, snaps):
        idx = math.floor((t - start).total_seconds() / bucket)
        buckets.setdefault(idx, []).append(s)
    target = size / len(buckets)
    cash = filled = 0.0
    for grp in buckets.values():
        pxs = [v.px for s in grp for v in s.venues]
        cash += min(target, size - filled) * (sum(pxs) / len(pxs))
        filled += target
    return cash, cash / size, None

# draws a random float uniformly
def _logu(rng, lo, hi):
    return 10 ** rng.uniform(math.log10(lo), math.log10(hi))

# search for the best parameter using randomized trials
def search_parameters(snaps: List[Snapshot], *, trials=120, seed=42):
    rng = np.random.default_rng(seed)
    best = {"cost": float("inf")}
    for _ in range(trials):
        lo = _logu(rng, 1e-3, 10)
        lu = _logu(rng, 1e-3, 10)
        th = _logu(rng, 1e-2, 1e2)
        cost, avg, _ = run_backtest(snaps, lo, lu, th)
        if cost < best["cost"]:
            best = {
                "cost": cost,
                "avg": avg,
                "lambda_over": lo,
                "lambda_under": lu,
                "theta_queue": th,
            }
    return best

# Calculate basis points improvement
def _bps(router_cost, base_cost):
    return (base_cost - router_cost) / base_cost * 1e4

# Full evaluation of router vs. baselines
def run_full_backtest(csv: str | Path = "l1_day.csv", *, trials: int = 120):
    """High-level helper: returns the full result dict (no printing)."""
    snaps = load_snapshots(csv)
    best = search_parameters(snaps, trials=trials)

    rc, ra, _ = run_backtest(
        snaps, best["lambda_over"], best["lambda_under"], best["theta_queue"]
    )
    bc, ba, _ = best_ask_baseline(snaps)
    tc, ta, _ = twap_baseline(snaps)
    vc, va, _ = vwap_baseline(snaps)

    return {
        "best_params": {
            k: best[k] for k in ("lambda_over", "lambda_under", "theta_queue")
        },
        "router": {"total_spent": rc, "avg_price": ra},
        "best_ask": {
            "total_spent": bc,
            "avg_price": ba,
            "savings_bps": _bps(rc, bc),
        },
        "twap": {
            "total_spent": tc,
            "avg_price": ta,
            "savings_bps": _bps(rc, tc),
        },
        "vwap": {
            "total_spent": vc,
            "avg_price": va,
            "savings_bps": _bps(rc, vc),
        },
    }

if __name__ == "__main__":
    res = run_full_backtest()
    print(json.dumps(res, indent=2))


{
  "best_params": {
    "lambda_over": 1.246878665907564,
    "lambda_under": 0.05695262671673521,
    "theta_queue": 27.188902645221663
  },
  "router": {
    "total_spent": 1114102.2800000003,
    "avg_price": 222.82045600000006
  },
  "best_ask": {
    "total_spent": 1114102.2800000003,
    "avg_price": 222.82045600000006,
    "savings_bps": 0.0
  },
  "twap": {
    "total_spent": 1115427.5850538702,
    "avg_price": 223.08551701077405,
    "savings_bps": 11.881587577968627
  },
  "vwap": {
    "total_spent": 1115320.7351223256,
    "avg_price": 223.06414702446511,
    "savings_bps": 10.924706086377022
  }
}
